In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from copy import copy

In [5]:
MOVIE_LENS_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'raw', 'movie_lense'))
MOVIE_LENS_PATH

'/home/levin/escaping_echo_chambers/data/raw/movie_lense'

In [9]:
genome_scores = pd.read_csv(os.path.join(MOVIE_LENS_PATH, 'genome-scores.csv'))

In [35]:
wide_genome_scores = genome_scores.pivot(index='movieId', columns='tagId')
wide_genome_scores.columns = ['TagId_' + str(i) for i in wide_genome_scores.columns.droplevel().rename(None)]

In [37]:
wide_genome_scores.head()

,TagId_1,TagId_2,TagId_3,TagId_4,TagId_5,TagId_6,TagId_7,TagId_8,TagId_9,TagId_10,...,TagId_1119,TagId_1120,TagId_1121,TagId_1122,TagId_1123,TagId_1124,TagId_1125,TagId_1126,TagId_1127,TagId_1128
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,0.25200,0.22750,0.02400,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
2,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,0.10100,0.08225,0.05250,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
3,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,0.14150,0.04075,0.03200,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
4,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,0.07475,0.03750,0.02400,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
5,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,0.07375,0.02825,0.02375,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575


In [47]:
def visualize_embedding(method, X, hyperparams, clusters, **kwargs):
    """
    Vizualizes grid of 2D embeddings varying hyperparams. Visualizes 2D and 3D embeddings with default parameters separately.
    
    Parameters
    ----------
    method : sklearn class
        embedding method with well-defined .fit_transform method, usually from sklearn.manifold
    X : np.array
        data
    method : sklearn class
        embedding method with well-defined .fit_transform method, usually from sklearn.manifold
    hyperparams : dict
        dictionary with hyperparams to test as keys and their respective ranges as values
    clusters : np.array
        Array of colors,ust be of the same shape with X. Usually this would be clustering colors
    kwargs : dict
        hyperparams of the method (including the default values for the ones to vary)

    Returns
    -------

    Notes
    -------
   
    """
    sns.set_style("whitegrid", {'axes.grid' : False})
    
#     if cbar == 'clustering':
# #         friendly_cmap = ListedColormap(sns.color_palette('colorblind', len(np.unique(colors))).as_hex())
#         almost_sequential_pal = configurations['clustering_palette']#['#1f78b4','#a6cee3','#fdbf6f','#ff7f00', '#cc78bc']
#         friendly_cmap = ListedColormap(sns.color_palette(almost_sequential_pal, len(np.unique(colors))).as_hex())
#     if cbar == 'colormap':
#         friendly_cmap = sns.cubehelix_palette(light=1, as_cmap=True)
#     if cbar is None:
#         friendly_cmap = None
    #TODO add scaling
    #Default params:
    default_args = copy(kwargs)
    embeddings_results = {} #dict with resulting embeddings with keys -- varied hyperparams, values: different embs
    for range_key in hyperparams.keys():
        #Set to default kwargs:
        kwargs = copy(default_args)
        X_embs = []
        for hyperparam in hyperparams[range_key]:
            print(range_key, '=', hyperparam)
            kwargs[range_key] = hyperparam
            model = method(**kwargs)
            # model.fit(X)
            X_embs.append(model.fit_transform(X))
        embeddings_results[range_key] = X_embs
        
    for key in embeddings_results.keys():
        X_embs = embeddings_results[key]
        fig, axs = plt.subplots(2, len(X_embs)//2, figsize=(30,15))#, #sharex=True, sharey=True, figsize=(20,20))
        fig.suptitle(method.__name__)
        plt.subplots_adjust(top=0.93)
        for num, ax_ in enumerate(axs.flatten()):
            ax_.scatter(X_embs[num][:, 0], X_embs[num][:, 1],  c = clusters)
            ax_.title.set_text('{} {}'.format(key, hyperparams[key][num]))

    
    kwargs = copy(default_args) #set to default
    kwargs['n_components'] = 2 #set to two to plot with default
    model = method(**kwargs)
    # model.fit(X)
#     if ax is None:
#         if load_path is None:
#             X_2D_emb = model.fit_transform(X)
#             if save_path is not None:
#                 np.save(os.path.join(save_path, 'X_2D_emb.npy'), X_2D_emb)
#         else:
#             X_2D_emb = np.load(os.path.join(load_path, 'X_2D_emb.npy'))
        # current_palette = sns.color_palette()
        # sns.palplot(current_palette)
        
#         fig = plt.figure(figsize = (15,10))
#         ax_ = fig.add_subplot(111)#, projection='3d')

#         scatter_2d = ax_.scatter(X_2D_emb[subselect,0], X_2D_emb[subselect, 1], c=colors, alpha = alpha, cmap = friendly_cmap)
# #         if cbar == 'clustering':
#             ticks = np.sort(np.unique(colors))[::-1]
#             cbar_clust = fig.colorbar(scatter_2d, ticks = ticks)
#             # print('CHECK CBAR:', np.sort(np.unique(colors))[::-1])
#             cbar_clust.ax.set_yticklabels(['Cluster ' + str(i) for i in ticks])
#         if cbar == 'colormap':
#             fig.colorbar(scatter_2d)
#         ax_.set_xlabel('{} 0'.format(method.__name__))
#         ax_.set_ylabel('{} 1'.format(method.__name__))
#         ax_.set_title('{} 2D projection'.format(method.__name__))
#         print('2D Args:', kwargs)
        
        # plt.savefig(IMG_PATH+'Viz_2D_{}.png'.format(method.__name__), bbox_inches = 'tight')
#         if filename is not None:
#             fig.savefig(filename+'_2D.png', bbox_inches = 'tight')

    ########3D:##########
#     kwargs = copy(default_args) #set to default
#     kwargs['n_components'] = 3
#     model = method(**kwargs)
    # model.fit(X)
#     if load_path is None:
#         X_3D_emb = model.fit_transform(X)
#         if save_path is not None:
#             np.save(os.path.join(save_path, 'X_3D_emb.npy'), X_3D_emb)
#     else:
#         X_3D_emb = np.load(os.path.join(load_path, 'X_3D_emb.npy'))
    
#     if ax is None:
#         fig = plt.figure(figsize = (15,10))
#         ax_ = fig.add_subplot(111, projection='3d')

#         ax_.scatter(X_3D_emb[subselect,0], X_3D_emb[subselect, 1], X_3D_emb[subselect, 2], c=colors, alpha = alpha, cmap = friendly_cmap)

#         ax_.set_xlabel('{} 0'.format(method.__name__))
#         ax_.set_ylabel('{} 1'.format(method.__name__))
#         ax_.set_zlabel('{} 2'.format(method.__name__))
#         ax_.set_title('{} 3D projection'.format(method.__name__))
#         ax_.view_init(30, 200)
#         print('3D Args:', kwargs)
        
#         # plt.savefig(IMG_PATH+'Viz_3D_{}.png'.format(method.__name__), bbox_inches = 'tight')
#         if filename is not None:
#             fig.savefig(filename+'_3D.png', bbox_inches = 'tight')
    
#     if ax is not None:
#         ax.scatter(X_3D_emb[subselect,0], X_3D_emb[subselect, 1], X_3D_emb[subselect, 2], c=colors, alpha = alpha, cmap = friendly_cmap)
#         ax.set_xlabel('{} 0'.format(method.__name__))
#         ax.set_ylabel('{} 1'.format(method.__name__))
#         ax.set_zlabel('{} 2'.format(method.__name__))
#         ax.set_title('{} 3D projection'.format(method.__name__))
#         # ax.view_init(30, 200)
#         print('3D Args:', kwargs)
#         return None, X_3D_emb


    return 

In [50]:
from sklearn.manifold import TSNE

method = TSNE
hyperparams = {'perplexity': [5, 20, 30, 50],
                      'early_exaggeration': [10.0, 12.0, 15.0, 20.0]}

visualize_embedding(method, wide_genome_scores, hyperparams, None, **{'perplexity': 30, 'early_exaggeration': 20.0,
 'n_components': 2,
 'n_iter': 300,
 'n_jobs': -1})

perplexity = 5


KeyboardInterrupt: 

In [51]:
from sklearn.manifold import Isomap

method = Isomap
hyperparams = {'n_neighbors': [5, 10, 20, 30, 40, 50, 100, 200]}

visualize_embedding(method, wide_genome_scores, hyperparams, None, **{'n_neighbors': 50,
 'n_components': 2,
 'max_iter': 1000,
 'n_jobs': -1})

n_neighbors = 5


KeyboardInterrupt: 

In [ ]:
from sklearn.manifold import SpectralEmbedding


method = SpectralEmbedding
hyperparams = {'n_neighbors': [5, 10, 20, 30, 40, 50, 100, 200]}

visualize_embedding(method, wide_genome_scores, hyperparams, None, **{'n_neighbors': 50,
 'n_components': 2,
 'n_jobs': -1})

n_neighbors = 5
